In [1]:
import re
import os
import json
import nltk
import math
import random
import unicodedata

from datetime import datetime

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException


class UNDETECTED(Exception):
    pass

TARGET_CLASS = "css-1422juy";
COMP_STATS = {"website":"", "name":"", "address":"", "reviews":-1, "keywords":""}

#COMPSTATS = {"name": "","category": "","address": "","phone": "","price_range": "","health_rating": "","info":"","working_hours": "","ratings": "","ratings_histogram": "","claimed_status": "","reviews": "","website": "","url":""}


PROTOCAL = "https://"
DOMAIN = "www.yelp.com"
URL = PROTOCAL+DOMAIN;


def parse(destination, location):

    global URL
    URL += "/search?find_desc="+destination+"&"+"find_loc="+location

driver = webdriver.Firefox()

In [2]:
parse("Museums", "San Jose, CA")
print(URL)

driver.get(URL)

https://www.yelp.com/search?find_desc=Museums&find_loc=San Jose, CA


In [3]:
T = driver.find_elements(By.XPATH, '//a[@class="'+TARGET_CLASS+'"]')

specific_urls = [link.get_attribute('href') for link in T]
names = [link.get_attribute('name') for link in T]

#    rI = [link.text for link in rInf]

print(specific_urls)
print(names)

['https://www.yelp.com/adredir?ad_business_id=VQ8hnL0N1OxWzktRGQQS7Q&campaign_id=BN69E-pvPqJpaMo0OPakvA&click_origin=search_results&placement=above_search&placement_slot=0&redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Frengstorff-house-mountain-view&request_id=809a820e01499c8e&signature=fb8b5b73947aaa464e38206a85a08d643f01dbdd031ee863c3b8b32c3d078949&slot=0', 'https://www.yelp.com/biz/rosicrucian-egyptian-museum-san-jose?osq=Museums', 'https://www.yelp.com/biz/the-tech-interactive-san-jose?osq=Museums', 'https://www.yelp.com/biz/childrens-discovery-museum-san-jose?osq=Museums', 'https://www.yelp.com/biz/san-jose-institute-of-contemporary-art-san-jose?osq=Museums', 'https://www.yelp.com/biz/winchester-mystery-house-san-jose?osq=Museums', 'https://www.yelp.com/biz/san-jose-museum-of-art-san-jose?osq=Museums', 'https://www.yelp.com/biz/beyond-van-gogh-san-jose?osq=Museums', 'https://www.yelp.com/biz/computer-history-museum-mountain-view?osq=Museums', 'https://www.yelp.com/biz/triton-muse

In [4]:
# for i in range(len(specific_urls)):
#    driver.get(specific_urls[i])
#        stick_info_div = driver.find_element(By.CLASS_NAME, " stickySidebar--fullHeight__09f24__kqHVd arrange-unit__09f24__rqHTg arrange-unit-grid-column--4__09f24__P05hD padding-l2__09f24__kf_t_ border-color--default__09f24__NPAKY");
driver.get(specific_urls[2])

In [65]:
#rInf = driver.find_elements(By.XPATH, '//div[@class="stickySidebar--fullHeight__09f24__kqHVd arrange-unit__09f24__rqHTg arrange-unit-grid-column--4__09f24__P05hD padding-l2__09f24__kf_t_ border-color--default__09f24__NPAKY"]')
rInf = driver.find_elements(By.XPATH, '//a[@class="css-10y60kr"]')
links= driver.find_elements(By.XPATH, '//a[@class="css-1422juy"]')

 

In [66]:
N = []

for i in range(len(links)):
    N.append(links[i].text)


L = [link.get_attribute('href') for link in links]

p = N.index('Ask a question')
N, L = N[0:p], L[0:p]


print(p)

5


In [67]:
area_data = {
    "name"	: "",
    "website" : "",  	
    "open_time" : "",
    "address" : "" ,
    "number" : "",
}


#f = driver.findElement(By.xpath("//div[@class='game-date-time']/span[@data-behavior='date_time']")).getAttribute("data-date");\

f = driver.find_element(By.XPATH,"//div[@class=' stickySidebar--fullHeight__09f24__kqHVd arrange-unit__09f24__rqHTg arrange-unit-grid-column--4__09f24__P05hD padding-l2__09f24__kf_t_ border-color--default__09f24__NPAKY']")
name = driver.find_element(By.XPATH,"//h1[@class='css-1x9iesk']")
sub = driver.find_elements(By.XPATH,"//span[@class=' css-1yy09vp']")

area_data["name"] = name.text

for comp in sub:
    if "reviews" in comp.text:
        area_data["reviews"] = comp.text.replace(" reviews", "")
    elif ( "AM" in comp.text ) or ( "PM" in comp.text ):
        area_data["open_time"] = comp.text
    else:
        continue

print(area_data)

{'name': 'The Tech Interactive', 'website': '', 'open_time': '10:00 AM - 3:00 PM', 'address': '', 'number': '', 'reviews': '694'}


In [76]:
print(f)
props = f.find_elements(By.XPATH, "//div[@class=' css-1vhakgw border--top__09f24__exYYb border-color--default__09f24__NPAKY']");
micro_links = f.find_elements(By.XPATH, "//a[@class='css-10y60kr']")
print(len(v)," ", len(micro_links))

<selenium.webdriver.remote.webelement.WebElement (session="6cabaf17-99be-4b1d-901d-a4f1ab9261b5", element="66d6fe3e-d2bc-40ce-893b-0bc7d3c4e245")>
3   10


In [82]:
def phone_number(n) -> bool:
    ns = ["0","1","2","3","4","5","6","7","8","9"]

    nc = 0

    for i in range(len(n)):
        if n[i] in ns:
            nc+=1
        
    return nc>8


for prop in props:
    T = prop.text

    if "http" in T:
        area_data["website"] = T
        continue

    if phone_number(T):
        area_data["number"] = T
        continue

    if "Get Directions" in T:
        TL = T.split("\n")
        area_data["address"] = TL[1]
        continue


area_data

{'name': 'The Tech Interactive',
 'website': 'http://www.thetech.org',
 'open_time': '10:00 AM - 3:00 PM',
 'address': '201 S Market St San Jose, CA 95113',
 'number': '(408) 294-8324',
 'reviews': '694'}